In [ ]:
import numpy as np 

import matplotlib.pyplot as plt

from delta_sigma_simulator.modulator import DeltaSigmaModulator
from delta_sigma_simulator.filter import FilterIntegrator, FilterButterworth
from delta_sigma_simulator.quantizer import QuantizerDelayHysteresis

In [ ]:
# Example of a 1st-order ASDM with: 
# - Loop filter
loop_filter = FilterIntegrator(1e9)
# - Quantizer
quantizer = QuantizerDelayHysteresis(25e-12, 0.0)
# - SDM
asdm = DeltaSigmaModulator(loop_filter, quantizer)

In [ ]:
# Simulation
# - Input frequency
fi = 100e6
# - Cut-off frequency
fc = 4 * fi
# - Sampling frequency
fs = 8 * fc
# - Output filter
output_filter = FilterButterworth(10, fc)
# - Time vector
t = 4.0 / fi + np.arange(0.0, 32.0 / fi - 0.5 / fs, 1.0 / fs)
# - Simulate
v = asdm.simulate([0.0, 0.1], fi, t, filter=output_filter)

In [ ]:
w = np.ones(len(v))

w1 = np.linalg.norm(w, 1)
w2 = np.linalg.norm(w, 2)

f = np.fft.rfftfreq(len(v), d=1.0 / fs)
V = np.fft.rfft(w * v) / (w1 / 2)

f_bin = int(fi / (fs / len(v)))

thd = 10 * np.log10(np.abs(V[3 * f_bin]) ** 2 / np.abs(V[1 * f_bin]) ** 2)

In [ ]:
plt.title(f'THD = {thd:.2f} dB')
plt.semilogx(f, 20 * np.log10(np.abs(V)), 'b-')   
plt.semilogx(f[1 * f_bin], 20 * np.log10(np.abs(V[1 * f_bin])), 'bx')
plt.semilogx(f[3 * f_bin], 20 * np.log10(np.abs(V[3 * f_bin])), 'bx')
plt.grid(which='both')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Magnitude [dBFS]')
plt.show()